In [2]:
import bs4
import requests
import json
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
import selenium

ModuleNotFoundError: No module named 'selenium'

The following code uses Firefox, if you use another browser, don't forget to adjust the below code with the browser of your choice

In [2]:
url = 'https://fln.dk/da/Praksis'
browser  = webdriver.Firefox()
browser.get(url)

browser.find_element_by_id("SearchSubmitButton").click()
browser.find_element_by_link_text("Alle resultater").click()

/tmp/ipykernel_284323/1834746117.py:5: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  browser.find_element_by_id("SearchSubmitButton").click()
/tmp/ipykernel_284323/1834746117.py:6: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  browser.find_element_by_link_text("Alle resultater").click()


Now, a new window will open in the browser specified. It can take up to a minute until "Alle resultater" is not underlined anymore

In [3]:
soup=bs4.BeautifulSoup(browser.page_source, 'html.parser')
cases = soup.find_all("p", {"class": "lineover"})
print("Extracted cases: ", len(cases))

Extracted cases:  9301


Extracting cases for the html object into python objects:

In [15]:
case_list = []
count = 0

for id, case in enumerate(cases):
    item = {}
    #strongs contain the 'emneord', these are the asylum motives
    strongs = case.find_all('strong')
    emne_raw = strongs[0].get_text().strip('\n')
    cleaned_asylmotive_string = " ".join(emne_raw.split()).strip()
    asylmotive_list = cleaned_asylmotive_string.replace('Emneord: ', '').split(',')
    
    #extracting country of origin
    country_raw = strongs[1].get_text()
    country = country_raw.replace('Land: ', '')
    
    #cleaning up the summary, there are a lot of empty lines
    text_raw = case.get_text().split('\n')
    for string in text_raw:
        if len(string) > 300:
            text = string
            count = count + 1
            hasText = True
            break
    
    item['id'] = id
    item['asylum_motive'] = asylmotive_list
    item['country'] = country
    item['text'] = text
    item['hasText'] = hasText
    case_list.append(item)
    
print("Cases that are stored in a python list now:", len(case_list))
print("Of the cases in the list, how many have a text summary:", count)

Cases that are stored in a python list now: 9301
Of the cases in the list, how many have a text summary: 8861


In [16]:
with open('data/dk_asylum_data.json', 'w') as file:
    json.dump(case_list, file)